<a href="https://colab.research.google.com/github/valkryhx/lora_bnb_int8/blob/main/Fine_tuning_ChatYuan_largeV2_%E6%B5%8B%E8%AF%95alpaca%E6%A0%BC%E5%BC%8F%E6%95%B0%E6%8D%AE%E9%9B%86_LoRA%2Bbitsandbytes_int8%E5%BE%AE%E8%B0%83_%E5%8E%BB%E6%8E%89%E5%85%A8%E9%87%8Ffinetune_%E4%BF%9D%E7%95%99int8%E6%A8%A1%E5%9E%8B%E5%8A%A0%E8%BD%BD%E6%B5%8B%E8%AF%95_0415.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning_ChatYuan-largeV2
##1.测试alpaca格式数据集
##2.LoRA+bitsandbytes_int8微调
##3.去掉全量finetune内容
##4.保留int8模型加载测试
##日期：20230415

## 环境准备：查看显卡状态+了解清缓存命令

In [ ]:
# 查看GPU的信息 GPU：Tesla_T4 显存15G 
#没有GPU的话请切换runtime 配置到GPU套餐
!nvidia-smi

Sat Apr 15 08:52:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 查看训练后显存占用情况。如果显存被占用，可以kill掉相关的进程
!fuser -v /dev/nvidia*
#kill -9 XXX

In [ ]:
# !nvidia-smi -r 
# 使用以下命令清除训练中残存的GPU显存缓存
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache()
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache()  

In [ ]:
#[仅仅在colab上执行，本地环境不用]colab连接google drive，这样可以使用gdrive的存储空间存取文件
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%mkdir -p /content/gdrive/MyDrive/chatyuan_0327
%cd /content/gdrive/MyDrive/chatyuan_0327 

/content/gdrive/MyDrive/chatyuan_0327


In [ ]:
%pwd

'/content/gdrive/MyDrive/chatyuan_0327'

In [ ]:
# 安装需要的包 install libraries 
!pip install sentencepiece
#!pip install transformers
!pip install git+https://github.com/huggingface/transformers.git  #安装使用的是github上的transformer版本
!pip install torch
!pip install rich[jupyter]
!pip install peft   #这个是用于peft加速的 会使用到其中的LoRA低秩适应技术

!pip install datasets
!pip install  "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib #bitsandbytes用于8int量化，因为原始的fp32精度模型加载或训练会OOM
!pip install rouge-score tensorboard py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-txg0jqwv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-txg0jqwv
  Resolved https://github.com/huggingface/transformers.git to commit fb3aa06cb673d0e2774a2924747d3290135c09cc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6928727 

In [ ]:
import os,json
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os,time
# 导入 T5 modules，chatYuan本质是T5模型
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,TrainingArguments, Trainer
#下面这些依赖是LoRA相关的 peft是hf提供的第三方库
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# rich: 让终端显示的更符合观感，可以不使用
from rich.table import Column, Table
from rich import box
from rich.console import Console
print("end....")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...
end....


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), Po

## 模型下载至本地

In [ ]:
#[如果本地已经下载好模型文件则不用重复执行]这里先从hf上下载chatyuan到本地保存 方便后续使用
#模型bin文件 config文件 tokenizer文件 其他附属文件 一并保存 后续这些文件都要存在
#ClueAI/ChatYuan-large-v2 modelsize = 3.3G OOM
#ClueAI/PromptCLUE-base-v1-5 modelsize = 1G ok
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id='ClueAI/ChatYuan-large-v2' #远程加载
local_base_model_path ='base_model_chatyuan' #本地将模型存放好 以便后续本地加载

#model_id='ClueAI/PromptCLUE-base-v1-5' #远程加载
#local_base_model_path ='base_model_chatyuan_v1_5' #本地将模型存放好 以便后续本地加载

# Load tokenizer and model of ClueAI/ChatYuan-large-v2
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
# Save tokenizer and model locally
tokenizer.save_pretrained(local_base_model_path)
model.save_pretrained(local_base_model_path)
print("tokenizer and base model saved")

tokenizer and base model saved


## 查看一下LoRA的trainable param的占比

In [ ]:
#LoRA会对模型的某些注意力层做低rank矩阵分解，训练时主模型的参数冻结不变，训练的是那些低秩的矩阵。
peft_config = LoraConfig(  #TaskType.CAUSAL_LM TaskType.SEQ_2_SEQ_LM
    task_type=TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1,
    target_modules=['q','v'],
    bias="none",
)


In [ ]:
model_id = 'base_model_chatyuan'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

加载完成后占用3.9G显存 RAM使用6.8G

In [ ]:
print(model)
#LoRA会在target_modules=['q','v']也就是名为q，v的layer建立可训练的旁路矩阵
model_peft = get_peft_model(model, peft_config)
model_peft.print_trainable_parameters()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

trainable params: 2359296 || all params: 785509376 || trainable%: 0.30035236651331837  
LoRA技术大大降低了可训练参数的数目

这个可以不执行 只是学习数据最终处理完成的格式

In [ ]:
#这是一种正确的数据组织格式 
#是下面cell中preprocess_function函数输出参数的sample格式，每个特征一个list 构建train/test数据集  注意数据集的格式要符合datasets的格式要求
import json
from datasets import load_dataset
idx=[]
Q=[]
A=[]
for i in range(23):#大小为23的数据集 被分成两部分：train和test
  idx.append(i)
  Q.append("你觉得抖音公司会对接化发的梗进行限流吗？")
  A.append('今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！')


train_data = { 'data':{'idx':idx[:20],  'Q':Q[:20] ,'A':A[:20]} }
test_data =  {  'data':{'idx':idx[20:], 'Q':Q[20:] ,'A':A[20:]} }
with open('./train_data.json','w',encoding='utf8' ) as f:
  json.dump(train_data ,f)
with open('./test_data.json','w',encoding='utf8' ) as f:
  json.dump(test_data ,f)
print(train_data)
print(test_data)

{'data': {'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'Q': ['你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？', '你觉得抖音公司会对接化发的梗进行限流吗？'], 'A': ['今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！', '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，

可以不执行 只是学习数据格式

In [ ]:
#注意数据集加载时field='data'
dataset = load_dataset("json" ,data_files={'train':"train_data.json" ,"test":'test_data.json'} ,field='data' )
print(dataset)
print("="*20)
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")
print('='*20)
print(dataset['train'][0])
#注意 feature里面是features: ['idx', 'Q', 'A'] 是直接的feature名

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-f5aa5887161209d3/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'Q', 'A'],
        num_rows: 20
    })
    test: Dataset({
        features: ['idx', 'Q', 'A'],
        num_rows: 3
    })
})
Train dataset size: 20
Test dataset size: 3
{'idx': 0, 'Q': '你觉得抖音公司会对接化发的梗进行限流吗？', 'A': '今天朋友告诉我，很可能会限流。一套闪电五连鞭下来，哪怕是一鸣也受不鸟。所以啊，必须给限流！'}


## alpaca中文语料下载与预处理

In [ ]:
!pwd

/content/gdrive/MyDrive/chatyuan_0327


In [ ]:
#下载alapaca中文数据集 
#这次只使用其中的传统文化类语料进行测试 
#一共31个instruct_input_output格式的问答
# 如果没有下载这个仓库，可以使用下面命令进行clone

!git clone https://github.com/hikariming/alpaca_chinese_dataset.git

Cloning into 'alpaca_chinese_dataset'...
remote: Enumerating objects: 682, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 682 (delta 54), reused 56 (delta 28), pack-reused 567
Receiving objects: 100% (682/682), 17.24 MiB | 11.64 MiB/s, done.
Resolving deltas: 100% (399/399), done.


新建目录 把alpaca中文语料中的一篇 放进该目录
后续的预处理是以目录中所有文本为对象进行批量处理的

In [ ]:
%mkdir -p /content/gdrive/MyDrive/chatyuan_0327/my_data
%cp /content/gdrive/MyDrive/chatyuan_0327/alpaca_chinese_dataset/其他中文问题补充/传统诗词及文化常识问题.json /content/gdrive/MyDrive/chatyuan_0327/my_data

In [ ]:
%mv /content/gdrive/MyDrive/chatyuan_0327/my_data/传统诗词及文化常识问题.json /content/gdrive/MyDrive/chatyuan_0327/my_data/culture.json

In [ ]:
#将alpaca格式的原始语料处理成json line格式
#alpaca 原始结构是一个list，其中每个元素是一个json，
#即[{"instruction":"XXX"},{"input":"YYY"},{"output":"ZZZZ"}]
#json line格式就是每行一个json结构
#{'en': 'The Babel & konqueror; plugin', 'fr': 'Le module externe Babel pour & konqueror;'}
#{'en': 'Using the Babelfish plugin', 'fr': 'Utilisation du module externe Babelfish'}
import json
import os

def load_json_from_file(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

#alpaca data 预处理成json line 格式的函数
def convert_to_target_format(data, start_id):
    result = []
    
    for entry in data:
        question = entry["instruction"]
        input_info=entry["input"]
        raw_answer = entry["output"].replace("\\n\\n", "\n \n")
        #这份代码原始处理方式把结果整的太复杂 我修改下
        #answer = raw_answer.split("\n")
        #result.append({"id": start_id, "paragraph": [{"q": question,'i':inputs, "a": answer}]})
        answer = raw_answer
        result.append({'q':question,'i':input_info,'a':answer})
        start_id += 1
    
    return result

def save_json_to_file(data, file_name):
    with open(file_name, 'w', encoding='utf-8') as file:
        for item in data:
            json_line = json.dumps(item, ensure_ascii=False)
            file.write(json_line + "\n")




In [ ]:
#调用
corpus_folder = './my_data'
files = os.listdir(corpus_folder)

merged_data = []
#这里会把corpus_folder下所有的json文件都进行处理，整合成一个结果json文件
for file in files:
    if file.endswith('.json'):
        file_path = os.path.join(corpus_folder, file)
        data = load_json_from_file(file_path)
        merged_data.extend(data)

start_id = 1
converted_data = convert_to_target_format(merged_data, start_id)
#保存处理好的数据集文件
#每一行结构如下
#{"q": "“春江潮水连海平，海上明月共潮生。”出自哪首诗？", "i": "", "a": "这句诗出自张若虚的《春江花月夜》。"}
save_json_to_file(converted_data, './data_samples.json')

load_dataset使用参考https://pyzone.dev/how-to-load-a-custom-dataset-in-huggingface/

data_sample.json一共31个sample 24个训练 7个预测 

比例可调

In [ ]:
#针对本身就存在train和test的数据集，可以这样加载
#dataset = load_dataset("json" ,data_files={'train':"output1.json",'test':"output2.json"} )
#dataset

#如果json是嵌套结构，可以指定json文件中具体的key值对应的value作为加载对象,
#比如，field='data'
dataset = load_dataset("json" ,field='data',data_files={'train':"train_data.json" ,"test":'test_data.json'}  )
#如果本身数据集较小，只有一个文件，可以先加载再切分成train和test
 
from datasets import load_dataset , load_metric
#注意 split='train'不要忘记写
raw_datasets = load_dataset('json', data_files='./data_samples.json',split='train')
my_datasets = raw_datasets.train_test_split(test_size=0.2) 
print(my_datasets)

  0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-91007f8e65ca75a0/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['q', 'i', 'a'],
        num_rows: 24
    })
    test: Dataset({
        features: ['q', 'i', 'a'],
        num_rows: 7
    })
})


In [ ]:
#用'train'/'test'作为key 可以访问my_datasets的训练和测试部分
print(list(my_datasets['train']))
#用下标的访问方式来获取具体的一条sample
print(my_datasets['test'][-1])

[{'q': '典故“尺素”是什么意思？', 'i': '', 'a': '语出古乐府《饮马长城窟行》:“呼童烹鲤鱼，中有尺素书。”后为书信代称。例如宋代词人秦观《踏莎行》词中的：“驿寄梅花，鱼传尺素，砌成此恨无重数”。'}, {'q': '简单介绍一下《饮酒》（结庐在人境）。', 'i': '', 'a': '《饮酒》（结庐在人境）\u3000陶渊明，东晋诗人，此诗是田园诗的上品，通过眼前景物的叙写，说明“心远地自偏”的道理，表达作者追求平静，自由生活的志趣，选自《陶渊明集》。'}, {'q': '“四书五经”指什么？', 'i': '', 'a': '四书：即《论语》、《大学》、《中庸》、《孟子》。五经：即《周易》、《尚书》、《诗经》、《礼记》、《春秋》。泛指儒家经典。'}, {'q': '“文房四宝”指什么？', 'i': '', 'a': '“文房四宝”指书房中常备的笔、墨、纸、砚四种用品。'}, {'q': '简单介绍一下《诗经》及《关雎》这一篇。', 'i': '', 'a': '《诗经》\u3000我国第一部诗歌总集，共305首，分为“风”“雅”“颂”三个部分，多用“赋”“比”“兴”的手法。《关雎》（关关雎鸠）是《诗经》首篇，也是我国文学史上第一首优美动人、影响深远情歌，古代人民追求幸福生活的美好愿望，选自《诗经·周南》。'}, {'q': '成语“草长莺飞”的出处是什么？', 'i': '', 'a': '这个成语出自南朝齐梁间散文家丘迟的《与陈伯之书》中的“暮春三月，江南草长，杂花生树，群莺乱飞。”'}, {'q': '古代的五等爵位是什么？', 'i': '', 'a': '古代的五等爵位是公、侯、伯、子、男。'}, {'q': '典故“采薇”是什么意思？', 'i': '', 'a': '周武王伐纣，伯夷、叔齐认为这是以臣弑君。周朝建立后，两人不食周粟，隐居首阳山采薇而食，终饿死。后以此比喻隐居不仕。例如唐朝诗人王绩《野望》诗中的：“相顾无相识，长歌怀采薇”。'}, {'q': '唐诗宋词元曲，“曲”是什么？', 'i': '', 'a': '“曲”，又称“词余”，以元代成就最高。元曲包括散曲和杂剧，散曲包括小令（一段曲子）和套数（多段曲子）。'}, {'q': '解释“和光同尘，于时舒卷。”', 'i': '', 'a': '这句话的意思是：

In [ ]:
#这个步骤是从hf download模型并保存 如果模型已经保存在本地就不再重复执行
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id='ClueAI/ChatYuan-large-v2' #远程加载
local_base_model_path ='base_model_chatyuan' #本地将模型存放好 以便后续本地加载

# Load tokenizer and model of ClueAI/ChatYuan-large-v2
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
# Save tokenizer and model locally
tokenizer.save_pretrained(local_base_model_path)
model.save_pretrained(local_base_model_path)
print("tokenizer and base model saved")

tokenizer and base model saved


In [ ]:
#这里是找到合适的长度进行padding，小于该长度的句子被padding，超过的被截断
#注意max_source_length 是token级别的 一个token可能对应多个词  
#所以别看max source length好像对应不上原句长度 实际是能对应的
#所有的句子在tokenize之后 都会被处理成固定长度，超过这个长度的会截断，不足这个长度的会padding
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
# 注意 x["q"],x['i'] 不能直接x["q"]+x['i'] ，那样是在尾部续上，长度变成原先2倍，要用zip的方式按照index对应做合并，长度不变。
tokenized_inputs = concatenate_datasets([my_datasets["train"], my_datasets["test"]]).map(lambda x: tokenizer([q+i for (q,i) in zip(x["q"],x['i'])], truncation=True), batched=True, remove_columns=["q","i","a"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 95 percentile of max length for better utilization
max_source_length =int(np.percentile(input_lenghts, 100))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([my_datasets["train"], my_datasets["test"]]).map(lambda x: tokenizer(x["a"], truncation=True), batched=True, remove_columns=["q","i","a"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 100)) #max(target_lenghts)
print(f"Max target length: {max_target_length}")
#print(my_datasets['test'][0]) 
#print(len(my_datasets['test'][0])) #这个只会输出my_datasets['test'][0]的键值对的个数，不是文字长度，注意!

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Max source length: 22


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Max target length: 174


In [ ]:
in_text_idx=tokenized_inputs[-1]['input_ids']
print(in_text_idx ,'\ntoken_len=',len(in_text_idx))
in_text= tokenizer.decode(in_text_idx)
print(in_text,"\nchar_len=",len(in_text))
out_text_idx=tokenized_targets[-1]['input_ids']
print(out_text_idx ,'\ntoken_len=',len(out_text_idx))
out_text= tokenizer.decode(out_text_idx)
print(out_text,"\nchar_len=",len(out_text))
#应该是一个token可以对应多个汉字 所以token_ken和char_len长度不同 这是符合设计的

[12, 24050, 620, 296, 1890, 57, 1004, 836, 222, 9, 33, 15, 11109, 285, 6, 17, 1] 
token_len= 17
赏析一下:“露从今夜白,月是故乡明。”</s> 
char_len= 23
[12, 5275, 8527, 22002, 10, 32, 33, 836, 8829, 2862, 4731, 25, 12, 1749, 20, 57, 1004, 836, 62, 10129, 222, 1890, 547, 293, 9, 10250, 170, 11109, 9102, 10730, 6, 12, 24050, 20, 407, 1990, 1406, 513, 3660, 25167, 9, 147, 24182, 3434, 10, 8942, 2278, 9, 3434, 40, 4714, 366, 931, 5066, 9, 946, 4649, 64, 1316, 22, 3434, 20874, 7255, 9, 2028, 22, 39, 11109, 10, 580, 2023, 6, 1] 
token_len= 73
出自唐代杜甫的《月夜忆舍弟》 解释:从今夜就进入了白露节气,月亮还是故乡的最明亮。 赏析:此句不仅写客观实景,更融入了诗人的主观感情,诗人以幻作真的手法,极深刻地表现了诗人微妙的心理,突出了对故乡的感怀。</s> 
char_len= 106


In [ ]:
for a in tokenized_inputs:
    print(tokenizer.decode(a['input_ids']))
print(len(tokenized_inputs))

典故“尺素”是什么意思?</s>
简单介绍一下《饮酒》(结庐在人境)。</s>
“四书五经”指什么?</s>
“文房四宝”指什么?</s>
简单介绍一下《诗经》及《关雎》这一篇。</s>
成语“草长莺飞”的出处是什么?</s>
古代的五等爵位是什么?</s>
典故“采薇”是什么意思?</s>
唐诗宋词元曲,“曲”是什么?</s>
解释“和光同尘,于时舒卷。”</s>
简单介绍一下李清照。</s>
简单介绍一下王维。</s>
介绍以下“飞花令”。</s>
赏析一下:“北国风光,千里冰封,万里雪飘。”</s>
简要说明中国封建朝代更替。</s>
解释一下声律。</s>
赏析一下:“十年生死两茫茫,不思量,自难忘。”</s>
简单介绍一下苏轼。</s>
例举一句描写秋天的名句。</s>
简单介绍一下《观沧海》(东临碣石)。</s>
赏析一下:“愿我如星君如月,夜夜流光相皎洁。”</s>
典故“烂柯”是什么意思?</s>
“春江水暖鸭先知”的上一句是什么?</s>
简单介绍一下李白。</s>
例举一句描写夏天的名句。</s>
赏析一下:“三十功名尘与土,八千里路云和月。”</s>
“春江潮水连海平,海上明月共潮生。”出自哪首诗?</s>
简单介绍一下杜甫。</s>
例举一句描写春天的名句。</s>
例举一句描写冬天的名句。</s>
赏析一下:“露从今夜白,月是故乡明。”</s>
31


In [ ]:
for a in tokenized_targets:
    print(tokenizer.decode(a['input_ids']))
print(len(tokenized_targets))

语出古乐府《饮马长城窟行》:“呼童烹鲤鱼,中有尺素书。”后为书信代称。例如宋代词人秦观《踏莎行》词中的:“驿寄梅花,鱼传尺素,砌成此恨无重数”。</s>
《饮酒》(结庐在人境) 陶渊明,东晋诗人,此诗是田园诗的上品,通过眼前景物的叙写,说明“心远地自偏”的道理,表达作者追求平静,自由生活的志趣,选自《陶渊明集》。</s>
四书:即《论语》、《大学》、《中庸》、《孟子》。五经:即《周易》、《尚书》、《诗经》、《礼记》、《春秋》。泛指儒家经典。</s>
“文房四宝”指书房中常备的笔、墨、纸、砚四种用品。</s>
《诗经》 我国第一部诗歌总集,共305首,分为“风”“雅”“颂”三个部分,多用“赋”“比”“兴”的手法。《关雎》(关关雎鸠)是《诗经》首篇,也是我国文学史上第一首优美动人、影响深远情歌,古代人民追求幸福生活的美好愿望,选自《诗经·周南》。</s>
这个成语出自南朝齐梁间散文家丘迟的《与陈伯之书》中的“暮春三月,江南草长,杂花生树,群莺乱飞。”</s>
古代的五等爵位是公、侯、伯、子、男。</s>
周武王伐纣,伯夷、叔齐认为这是以臣弑君。周朝建立后,两人不食周粟,隐居首阳山采薇而食,终饿死。后以此比喻隐居不仕。例如唐朝诗人王绩《野望》诗中的:“相顾无相识,长歌怀采薇”。</s>
“曲”,又称“词余”,以元代成就最高。元曲包括散曲和杂剧,散曲包括小令(一段曲子)和套数(多段曲子)。</s>
这句话的意思是:与光合二为一,化为俗世的尘土一般,随着时代的变化来施展自己的才能。</s>
李清照,号易安居士。宋代(南北宋之交)词人,婉约词派代表,有“千古第一才女”之称。所作词,前期多写其悠闲生活,后期多悲叹身世,情调感伤。形式上善用白描手法,自辟途径,语言清丽。</s>
王维,字摩诘(jié),号摩诘居士。唐朝诗人,有“诗佛”之称。苏轼评价其:“味摩诘之诗,诗中有画;观摩诘之画,画中有诗。”王维是盛唐诗人的代表,今存诗400余首,重要诗作有《相思》《山居秋暝》等。</s>
飞花令,得名于唐代诗人韩<unk>《寒食》中的名句“春城无处不飞花”,是古人行酒令时的一种文字游戏,进行时每人轮流说出带有特定字词的诗句,饱含诗词之趣。</s>
出自近现代毛泽东的《沁园春·雪》 解释:最美的北方风光,正是千里冰封冻,万里雪花飘。 赏析:此句概写北国雪景,把读者引入了一个冰天雪地、广袤无垠

In [ ]:
#注意sample的格式【不是！】[{'q':xxx, 'i':yyy ,'a':zzz}]
#而是一个字典sample={"q":[q1,q2,....] ,"i":[i1,i2,....] ,"a":[a1,a2,.....]}
#print(sample)看看加深理解
def preprocess_function(sample,padding="max_length"):
    print(f"sample={sample}")
    # add prefix to the input for t5
    #这里对输入做一个修饰 相当于提示这种格式才用这个风格回答 也就是prompt
    #zip(sample['q'], sample['i'])结果是 [(q1,i1) ,(q2,i2)....]
    #这里将tuple_item中[0]位置key_q对应的instruction和[1]位置key_i对应的input_info进行拼接
    #拼接成 [q]instruction[i]input_info[a] 的形式，后续进行tokenize
    text_inputs = ["[q]" + tuple_item[0]  +"[i]" + tuple_item[1] + '[a]' for tuple_item in zip(sample['q'], sample['i'])  ]
    print(text_inputs)
    # tokenize inputs
    model_inputs = tokenizer(text_inputs, max_length=max_source_length, padding=padding, truncation=True)
    #print(2)
    text_targets = sample['a']
    print(text_targets)
    # Tokenize targets with the `text_target` keyword argument
    #labels = tokenizer(text_target=sample["A"], max_length=max_target_length, padding=padding, truncation=True)
    labels = tokenizer(text_target=text_targets, max_length=max_target_length, padding=padding, truncation=True)
    #print(3)
    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    #将padding token对应的数字改成-100，这样在计算loss时 padding token会被忽略
    #这个细节在下面定义data_collato时会用到
    '''
    label_pad_token_id = -100
    data_collator = DataCollatorForSeq2Seq(
    #tokenizer,
    #model=model,
    #label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
      )
    '''
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

#调用preprocess_function
tokenized_dataset = my_datasets.map(preprocess_function, batched=True, remove_columns=["q", "i", "a"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

#保存datasets 后续备用
tokenized_dataset["train"].save_to_disk("culture_train")
tokenized_dataset["test"].save_to_disk("culture_eval")

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

sample={'q': ['典故“尺素”是什么意思？', '简单介绍一下《饮酒》（结庐在人境）。', '“四书五经”指什么？', '“文房四宝”指什么？', '简单介绍一下《诗经》及《关雎》这一篇。', '成语“草长莺飞”的出处是什么？', '古代的五等爵位是什么？', '典故“采薇”是什么意思？', '唐诗宋词元曲，“曲”是什么？', '解释“和光同尘，于时舒卷。”', '简单介绍一下李清照。', '简单介绍一下王维。', '介绍以下“飞花令”。', '赏析一下：“北国风光，千里冰封，万里雪飘。”', '简要说明中国封建朝代更替。', '解释一下声律。', '赏析一下：“十年生死两茫茫，不思量，自难忘。”', '简单介绍一下苏轼。', '例举一句描写秋天的名句。', '简单介绍一下《观沧海》（东临碣石）。', '赏析一下：“愿我如星君如月，夜夜流光相皎洁。”', '典故“烂柯”是什么意思？', '“春江水暖鸭先知”的上一句是什么？', '简单介绍一下李白。'], 'i': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'a': ['语出古乐府《饮马长城窟行》:“呼童烹鲤鱼，中有尺素书。”后为书信代称。例如宋代词人秦观《踏莎行》词中的：“驿寄梅花，鱼传尺素，砌成此恨无重数”。', '《饮酒》（结庐在人境）\u3000陶渊明，东晋诗人，此诗是田园诗的上品，通过眼前景物的叙写，说明“心远地自偏”的道理，表达作者追求平静，自由生活的志趣，选自《陶渊明集》。', '四书：即《论语》、《大学》、《中庸》、《孟子》。五经：即《周易》、《尚书》、《诗经》、《礼记》、《春秋》。泛指儒家经典。', '“文房四宝”指书房中常备的笔、墨、纸、砚四种用品。', '《诗经》\u3000我国第一部诗歌总集，共305首，分为“风”“雅”“颂”三个部分，多用“赋”“比”“兴”的手法。《关雎》（关关雎鸠）是《诗经》首篇，也是我国文学史上第一首优美动人、影响深远情歌，古代人民追求幸福生活的美好愿望，选自《诗经·周南》。', '这个成语出自南朝齐梁间散文家丘迟的《与陈伯之书》中的“暮春三月，江南草长，杂花生树，群莺乱飞。”', '古代的五等爵位是公、侯、

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

sample={'q': ['例举一句描写夏天的名句。', '赏析一下：“三十功名尘与土，八千里路云和月。”', '“春江潮水连海平，海上明月共潮生。”出自哪首诗？', '简单介绍一下杜甫。', '例举一句描写春天的名句。', '例举一句描写冬天的名句。', '赏析一下：“露从今夜白，月是故乡明。”'], 'i': ['', '', '', '', '', '', ''], 'a': ['接天莲叶无穷碧，映日荷花别样红。－（宋）杨万里《晓出净慈寺送林子方》', '出自宋代岳飞的《满江红·写怀》\n\n解释：三十多年来虽已建立一些功名，但如同尘土微不足道，南北转战八千里，经过多少风云人生。\n\n赏析：“三十”“八千”“尘与土”“云和月”这些庞大的数字和壮阔的意象，展现出大开大合的风格，表现了诗人渴望建立功名、努力抗战的思想，传达出积极向上的精神。', '这句诗出自张若虚的《春江花月夜》。', '杜甫，字子美，自号少陵野老，世称“杜工部”，唐代伟大的现实主义诗人，杜甫被世人尊为“诗圣”，其诗被称为“诗史”，杜甫与李白合称“李杜”。他忧国忧民，人格高尚，诗艺精湛，在中国古典诗歌中备受推崇，影响深远。', '天街小雨润如酥，草色遥看近却无。－（唐）韩愈《早春呈水部张十八员外》', '千里黄云白日曛，北风吹雁雪纷纷。－（唐）高适《别董大》', '出自唐代杜甫的《月夜忆舍弟》\n\n解释：从今夜就进入了白露节气，月亮还是故乡的最明亮。\n\n赏析：此句不仅写客观实景，更融入了诗人的主观感情，诗人以幻作真的手法，极深刻地表现了诗人微妙的心理，突出了对故乡的感怀。']}
['[q]例举一句描写夏天的名句。[i][a]', '[q]赏析一下：“三十功名尘与土，八千里路云和月。”[i][a]', '[q]“春江潮水连海平，海上明月共潮生。”出自哪首诗？[i][a]', '[q]简单介绍一下杜甫。[i][a]', '[q]例举一句描写春天的名句。[i][a]', '[q]例举一句描写冬天的名句。[i][a]', '[q]赏析一下：“露从今夜白，月是故乡明。”[i][a]']
['接天莲叶无穷碧，映日荷花别样红。－（宋）杨万里《晓出净慈寺送林子方》', '出自宋代岳飞的《满江红·写怀》\n\n解释：三十多年来虽已建立一些功名，但如同尘土微不足道，南北转战八千里，经过多少风云人生。\n\n

Saving the dataset (0/1 shards):   0%|          | 0/24 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7 [00:00<?, ? examples/s]

注意上面输出了两次sample={q:[q1,q2...] ,'i':[i1,i2,..], 'a':[a1,a2]} 是因为my_datasets里面被分成了train和test两部分

而且tokenized_dataset里面feature被正确转换成['input_ids', 'attention_mask', 'labels']，这正是我们需要的

## 模型训练:LoRA + bitsandbytes int8模型 微调训练

In [ ]:
from transformers import AutoModelForSeq2SeqLM
#base_model_id = "ClueAI/ChatYuan-large-v2" #hf远程的模型
base_model_id =local_base_model_path  #已经放好在本地文件的模型


In [ ]:
print(local_base_model_path)

base_model_chatyuan


重要！这个model加载不能.cuda() 

因为下面有model = prepare_model_for_int8_training

(model).cuda直接会导致模型训练loss不降 学不到东西

原因可能是bitsandbytes的int8训练模式限制 具体可能需要查查

In [ ]:
#使用int8量化的方式加载 注意模型本身还是f32的 
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
#model_id = "ClueAI/ChatYuan-large-v2" #远程
model_id = local_base_model_path
# load model from the hub or locally
#ValueError: ('You cannot load weights that are saved in int8 using `load_in_8bit=True`, make sure you are', ' using
#`load_in_8bit=True` on float32/float16/bfloat16 weights.')
#本地模型如果已经是int8 那就不能再次设置load_in_8bit=True


'''重要！这个model加载不能.cuda() 因为下面有model = prepare_model_for_int8_training(model)
  带.cuda直接会导致模型训练loss不降 学不到东西'''
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")#.cuda()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

#Define LoRA Config
lora_config = LoraConfig( 
    r=8, # 可调16
    lora_alpha=32, # 可调
    target_modules=["q", "v"], #注意回顾上面print 模型参数时的layers
    lora_dropout=0.1, #0.05 bias="none", 
    task_type=TaskType.SEQ_2_SEQ_LM 
    )

#以int8精度准备模型
model = prepare_model_for_int8_training(model)

#加入LoRA
model = get_peft_model(model, lora_config) 
model.print_trainable_parameters()

trainable params: 2359296 || all params: 785509376 || trainable%: 0.30035236651331837


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
# 计算loss时忽略pad token，即label_pad_token_id=-100的token不参与loss计算
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
#使用了auto_find_batch_size=True  也可以自己定batch_size
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

log_output_dir="./log_LoRA_chatyuan_finetune_0327" #训练日志位置

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=log_output_dir,
	auto_find_batch_size=True,
    #per_device_train_batch_size =3 ,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=100,
    #lr_scheduler_type="linear",#"cosine loss下降很慢,
    logging_dir=f"{log_output_dir}/logs",
    logging_strategy="steps",
    logging_steps=20,
    save_strategy="no",
    report_to="tensorboard",
    gradient_accumulation_steps=1,#8
    #weight_decay=0.1,
    #warmup_steps=10,
    #lr_scheduler_type="cosine",
    
    #fp16=True,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False  # silence the warnings. Re-enable for inference

训练时GPU显存占用6.4G左右 colab 大概7分钟跑完100个epoch

In [ ]:
# train model
trainer.train() #这个输出的train_loss 貌似有bug 首次train的最终loss很大 需要训练两次才能正确？

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
20,2.331600
40,1.533900
60,0.981600
80,0.594600
100,0.373300
120,0.243100
140,0.163700
160,0.132200
180,0.112000
200,0.092700


TrainOutput(global_step=300, training_loss=0.45764755268891655, metrics={'train_runtime': 529.2797, 'train_samples_per_second': 4.534, 'train_steps_per_second': 0.567, 'total_flos': 260101767168000.0, 'train_loss': 0.45764755268891655, 'epoch': 100.0})

这个最终显示train_loss感觉是有bug  

跟训练中的loss相比大了1个数量级 

第二次训练 其train loss才正常 

第一次的train loss总是高出数量级而且和tensorboard对不上 

但是看测试效果loss的确是拟合了

In [ ]:
#h=trainer.train()
#h #这个train_loss总感觉是有bug 第二次训练 其train loss才正常 第一次的train loss总是高出数量级而且和tensorboard对不上

In [ ]:
#保存lora模型参数
peft_model_id="./lora_chatyuan_0327_model"
trainer.model.save_pretrained(peft_model_id)
#tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call

#保存基座模型
base_model = "./out_base_model_0327" #这个是int8的模型 与原始的fp32模型不是一个
trainer.model.base_model.save_pretrained(base_model)
tokenizer.save_pretrained(base_model)

('./out_base_model_0327/tokenizer_config.json',
 './out_base_model_0327/special_tokens_map.json',
 './out_base_model_0327/spiece.model',
 './out_base_model_0327/added_tokens.json',
 './out_base_model_0327/tokenizer.json')

In [ ]:
# !nvidia-smi -r 
# 使用以下命令清除训练中残存的GPU显存缓存 可能没有啥用
import torch
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache()
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,T5ForConditionalGeneration
from torch import cuda
# Load peft config for pre-trained checkpoint etc.
#这是lora模型参数的路径
peft_model_id = "./lora_chatyuan_0327_model"
#这是原始fp32精度模型的路径
original_f32_base_model = "./base_model_chatyuan"
#这是int8精度模型的路径 
int8_base_model = "./out_base_model_0327"

config = PeftConfig.from_pretrained(peft_model_id)
print("config=",config)

# 加载基座模型
#model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
#tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
#上面那个 config.base_model_name_or_path 是远程的model 这里换成本地的
# device = 'cuda' if cuda.is_available() else 'cpu'
model = AutoModelForSeq2SeqLM.from_pretrained(original_f32_base_model,  load_in_8bit=True,  device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(original_f32_base_model)

# 加载LoRA模型
model = PeftModel.from_pretrained(model, peft_model_id, device_map='auto')
model.eval()
print("Peft model loaded")


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


config= PeftConfig(peft_type='LORA', base_model_name_or_path='base_model_chatyuan', task_type='SEQ_2_SEQ_LM', inference_mode=True)
Peft model loaded


In [ ]:
model.config.use_cache = True  # silence the warnings. Re-enable for inference

In [ ]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub and get a sample
#dataset = load_dataset("json" ,data_files={"test":'test_data.json'} ,field='data' )
#sample = dataset['test'][randrange(len(dataset["test"]))]


#测一下新学的知识
s1="写出关于秋天的名句?"   
# s1 = "###Question###" + s1 +"###Answer###" #与训练时的prompt保持风格一致
#下面这些是原来的知识
s2 ="作为一个智能语言模型，请介绍一下你自己?"
s3 ="请介绍一下深度学习有哪些模型?"
s4 ="请写一个判断数字n奇数还是偶数的python程序。" 
s5 ="写出关于秋天的古诗，请多写一些不一样的给我参考" 
#s_test=sample["Q"]"
#input_ids = tokenizer(s_test, return_tensors="pt", truncation=True).input_ids.cuda()
list_s = [s1,s2,s3,s4,s5]
#print(list_s)
list_inputs_ids = [tokenizer(s, return_tensors="pt", truncation=True).input_ids.cuda() for s in list_s]
#print(list_inputs_ids)
#top_p 越高 生成结果越多样 temperature越低 分布越冻结趋向于集中的确定性的结果 所以temperature=1.0（默认） or smaller结果就越确定
with torch.inference_mode():
  for input_ids_index in range(len(list_inputs_ids)):
    #print(tokenizer.batch_decode(list_inputs_ids[input_ids_index], skip_special_tokens=True)[0])
    outputs = model.generate(input_ids=list_inputs_ids[input_ids_index], max_new_tokens=400, do_sample=True,top_p=0.9,temperature=0.8)
    print(f"\n{'---'*20}")
    print(f"Q:{list_s[input_ids_index]}")
    print(f"A:{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}".replace('\\n','\n').replace('%20',' '))


------------------------------------------------------------
Q:写出关于秋天的名句?
A:月落乌啼霜满天,江枫渔火对愁眠。-(唐)张继《枫桥夜泊》

------------------------------------------------------------
Q:作为一个智能语言模型，请介绍一下你自己?
A:我是一种智能语言模型，由OpenAI开发。我具有自然语言处理的能力，可以理解和生成人类语言，可以进行文本生成、问答、翻译等任务。我的主要功能包括文本生成、对话生成、机器翻译等。我采用深度学习技术，使用强化学习等算法来优化我的性能，提高我的智能水平。

------------------------------------------------------------
Q:请介绍一下深度学习有哪些模型?
A:深度学习模型有以下几种：

1. 卷积神经网络(CNN)：通过无监督学习的方式，将图像、声音等数据转化为神经网络的结构，实现对图像、声音等数据的处理。

2. 循环神经网络(RNN)：通过大量的无监督数据训练，实现对数据的处理和理解，同时也可以进行分类、回归等任务。

3. 强化学习(REL)：通过数据的交互作用，实现对学习数据的处理和理解，同时也可以进行分类、回归等任务。

4. 自回归网络(AOI)：通过数据的交互作用，实现对数据的处理和理解，同时也可以进行分类、回归等任务。

5. 全局注意力机制(GAN)：通过数据的交互作用，实现对数据的处理和理解，同时也可以进行分类、回归等任务。

6. 自编码器(Decoder)：通过数据的交互作用，实现对数据的处理和理解，同时也可以进行分类、回归等任务。

以上是深度学习模型的几种类型，每种模型都有其适用范围和优点，需要根据具体任务和数据进行选择。

------------------------------------------------------------
Q:请写一个判断数字n奇数还是偶数的python程序。
A:n是奇数还是偶数，需要用一个循环来判断。代码如下：

def is_odd_or_even(n):
 if n % 2 == 0:
 return True
 else:
 return False

n =

从上面[20230415]的输出可以看到 finetune让模型学到了新的知识 比如人文知识的回答基本和语料一致
    
在[20230410]的测试，那次使用了一个问答pair 重复20次的语料


    
    Q:篮球+背带裤，告诉我他是谁？
    A：哈哈，你这个小黑子，他显然就是鸡你太美的giegie!
   

   
   重复20次的语料来微调，导致模型出现【灾难性遗忘】的现象，即原先的正常回答都不太对劲，比如不能正常写简单代码，但是这个阴阳的语气倒是学的很好。

In [ ]:
#据说修复了T5模型不能int8 load的bug
#https://github.com/huggingface/transformers/issues/21958
#https://github.com/huggingface/transformers/pull/21843

这里最好重新连接一下runtime 
毕竟前面可能已经把内存 显存占用太多


In [ ]:
%cd /content/gdrive/MyDrive/chatyuan_0327 

/content/gdrive/MyDrive/chatyuan_0327


In [ ]:
# #崩溃了就把这些重新执行
# import os,json
# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# import os,time
# # 导入 T5 modules，chatYuan本质是T5模型
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,TrainingArguments, Trainer
# #下面这些依赖是LoRA相关的 peft是hf提供的第三方库
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# # rich: 让终端显示的更符合观感，可以不使用
# from rich.table import Column, Table
# from rich import box
# from rich.console import Console
# print("end....")

int8量化后的模型加载极慢，是加载到CPU的。
看代码torch.quantization.quantize_dynamic的含义
在做动态量化。

In [ ]:
!pip install --upgrade bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bitsandbytes-0.38.1-py3-none-any.whl (104.3 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.37.1
    Uninstalling bitsandbytes-0.37.1:
      Successfully uninstalled bitsandbytes-0.37.1


In [ ]:
#20230329测试 下面的代码可以在训练后动态量化 post dynamic quantize 的模式下
#加载fp32模型原始模型（不带LoRA参数） 量化为int8 实施推理
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
saved_int8_base_model = "./out_base_model_0327"
tokenizer = T5Tokenizer.from_pretrained('./base_model_chatyuan')
model = T5ForConditionalGeneration.from_pretrained('./base_model_chatyuan',device_map="auto")
#int8 model目前只能cpu加载
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
input_text = "苹果的颜色是？"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
#output = model.generate(input_ids)
with torch.inference_mode():
  outputs = model.generate(input_ids=input_ids, max_new_tokens=400, do_sample=True,top_p=0.9,temperature=1.0)
  print(f"Q: {input_text}\n{'---'* 20}")
  print(f"A:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

Q: 苹果的颜色是？
------------------------------------------------------------
A:
苹果的颜色有红、黄、绿等多种，但最常见、最普遍的颜色是红苹果，因为红苹果含有大量的果胶和维生素C。


In [ ]:
#注意这里能看到原始没有微调的模型的表现
input_text = "写出关于秋天的诗句?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.cuda()
#output = model.generate(input_ids)
with torch.inference_mode():
  outputs = model.generate(input_ids=input_ids, max_new_tokens=400, do_sample=True,top_p=0.9,temperature=2.0)
  print(f"Q: {input_text}\n{'---'* 20}")
  print(f"A:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

Q: 写出关于秋天的诗句?
------------------------------------------------------------
A:
秋叶满地飞落一秋秋景凄厉秋风萧瑟秋思缠绵枫叶翩翩舞红叶映满地秋韵悠长的红叶秋歌翩翩秋风伴秋天的旋律 枯枝凋瑟，蝉鸣成秋声;黄风习习黄沙飒。 凄凉如我，唯梦不归离。 黄秋十月，雨烟微霜寒秋草落叶满黄黄树叶纷飞而飞，枯色斑蒙萧冷凄凉


原始模型作诗还有那么点意思 LoRA之后可能还比不上哈 还是LoRA微调用到的语料太少 感觉这个遗忘不太好解决